<a href="https://colab.research.google.com/github/MHHamdan/LLM_Reasoning/blob/main/multiRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [5]:

#!pip install langchain openai faiss-cpu python-dotenv
#!pip install langchain_community


# Import dependencies

In [6]:
import os
from typing import List, Dict
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


# Set up environment variables

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = "sk-proj-MohEMWmXXXXXXXXXXXXXX"  # Replace with your API key


# Implement RetrievalRouterAgent

In [8]:
class RetrievalRouterAgent:
    def __init__(self, llm):
        self.llm = llm
        self.memory = ConversationBufferMemory(memory_key="chat_history")

    def route_query(self, query: str) -> List[str]:
        prompt = f"""Analyze this query and determine which data sources to search:
        Query: {query}
        Available sources: vector_db, web_search, email, chat
        Return only the source names separated by comma."""

        response = self.llm(prompt)
        return [source.strip() for source in response.split(',')]


# Implement RetrievalAgent

In [9]:
class RetrievalAgent:
    def __init__(self, source_type: str, tools: List[Tool]):
        self.source_type = source_type
        self.tools = tools

    async def retrieve(self, query: str) -> Dict:
        if self.source_type == "vector_db":
            return await self._vector_search(query)
        elif self.source_type == "web_search":
            return await self._web_search(query)

    async def _vector_search(self, query: str) -> Dict:
        embeddings = OpenAIEmbeddings()
        # Demo vector store with sample data
        texts = ["Sample document 1", "Sample document 2"]
        vector_store = FAISS.from_texts(texts, embeddings)
        results = vector_store.similarity_search(query)
        return {"source": "vector_db", "results": str(results)}

    async def _web_search(self, query: str) -> Dict:
        # Simplified web search simulation
        return {"source": "web_search", "results": f"Web search results for: {query}"}


# Implement MultiAgentRAG

In [10]:
class MultiAgentRAG:
    def __init__(self, llm):
        self.llm = llm
        self.router = RetrievalRouterAgent(llm)
        self.retrieval_agents = {
            "vector_db": RetrievalAgent("vector_db", []),
            "web_search": RetrievalAgent("web_search", []),
        }

    async def process_query(self, query: str) -> str:
        sources = self.router.route_query(query)

        results = []
        for source in sources:
            if source in self.retrieval_agents:
                result = await self.retrieval_agents[source].retrieve(query)
                results.append(result)

        return self._synthesize_results(query, results)

    def _synthesize_results(self, query: str, results: List[Dict]) -> str:
        context = "\n".join([f"From {r['source']}: {r['results']}" for r in results])
        prompt = f"""Based on the following information, answer the query:
        Query: {query}
        Context: {context}
        """
        return self.llm(prompt)



# Testing


In [11]:
async def test_system():
    llm = OpenAI(temperature=0)
    rag_system = MultiAgentRAG(llm)
    query = "What are the latest developments in AI?"
    response = await rag_system.process_query(query)
    print(f"Query: {query}\nResponse: {response}")


# Run the test

In [12]:
import asyncio
await test_system()

<ipython-input-11-f44870883a2c>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
<ipython-input-8-dbcdf4e7837d>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-8-dbcdf4e7837d>:12: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(prompt)


Query: What are the latest developments in AI?
Response: 
The latest developments in AI include advancements in natural language processing, computer vision, and machine learning algorithms. Some notable developments include the use of deep learning techniques for more accurate and efficient data analysis, the integration of AI into various industries such as healthcare and finance, and the development of AI-powered virtual assistants and chatbots. Additionally, there have been advancements in ethical considerations and regulations surrounding AI, as well as increased research and investment in the field.
